Continue working on the tree after basalv6. 
The aim is to remove rogue taxa and thereby hopefully increase overal support for the tree.

I'll remove:
* all picea abies
* Chlre_v5_5-Cre11_g467577_t1_1-Chlamydomonas_reinhardtii
* Equisetum
* tr_Q5KTX4_Q5KTX4_9VIRI_MADS-box_protein_CpMADS1__Fragment__OS_Closterium_peracerosum-strigosum-littorale_complex_OX_34146_GN_CpMADS1_PE_2_SV_1
* Pp014G086100
* XMGP-2008586-Juniperus_scopulorum

Finally, I will also remove all but one of the algal outgroup sequences, the solidity of this outgroup has been very reproducible.


I'm running trees initially with UFbootstrap and SH-alrt and later with a non-parametric tree inference.

I'm getting the basalv5 fasta and remove everything I don't want.

In [11]:
grep -v -f <(echo -e 'MA_\nChlre\nEquisetum\nQ5KTX4\nPp014G086100\nXMGP-2008586') \
    data/MIKC_orthogroup_selection-basal-v6_guide-v4.fasta \
    > data/MIKC_orthogroup_selection-basal-v7_guide-v4_TEMP.fasta

In [12]:
grep -c '>' data/MIKC_orthogroup_selection-basal-v6_guide-v4.fasta
grep -c '>' data/MIKC_orthogroup_selection-basal-v7_guide-v4_TEMP.fasta

188
171


And finally remove all but one of the algal outgroup sequences:

In [14]:
grep -v -f <(echo -e 'Q5KU24\nQ5KTX4\nMNNM\nQPDY\nFQLP\nTPHT\nTPHT') \
    data/MIKC_orthogroup_selection-basal-v7_guide-v4_TEMP.fasta \
    > data/MIKC_orthogroup_selection-basal-v7_guide-v4.fasta
rm data/MIKC_orthogroup_selection-basal-v7_guide-v4_TEMP.fasta

In [16]:
grep -c '>' data/MIKC_orthogroup_selection-basal-v7_guide-v4.fasta

165


In [17]:
inseq='MIKC_orthogroup_selection-basal-v7_guide-v4'

# aligning and trimming

In [18]:
conda activate phylogenetics
if    [ ! -d ./data/alignments_raw/ ]
then  mkdir  ./data/alignments_raw
fi
if    [ ! -f "./data/alignments_raw/$inseq"_aligned-mafft-linsi.fasta ]
then  einsi --thread $(nproc) data/$inseq.fasta > ./data/alignments_raw/"$inseq"_aligned-mafft-einsi.fasta
fi
conda deactivate

(phylogenetics) (phylogenetics) outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
All-to-all alignment.
tbfast-pair (aa) Version 7.407
alg=N, model=BLOSUM62, 2.00, -0.00, -0.00, noshift, amax=0.0
4 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
  160 / 165
done.

Progressive alignment ... 
STEP    88 /164 (thread    0) 
STEP    90 /164 (thread    1) done. *alloclen = 3899
STEP   164 /164 (thread    3) 
done.
tbfast (aa) Version 7.407
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
4 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 4
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16
Loading 'hat3' ... done.

  160 / 165
Segment   1/  1    1-3049
005-0326-0 (thread    3) worse         
Converged2.
done
dvtditr (aa) Version 7.407
alg=A, model=BLOSUM62, 1.53

In [20]:
conda activate phylogenetics
if    [ ! -d data/alignments_trimmed ]
then  mkdir  data/alignments_trimmed 
fi

gt=( 1 2 3 4 5 )

# define appendix only once here:
for g in ${gt[@]}
do  trimappendix=trim-gt"$g"
    for a in "data/alignments_raw/$inseq"_aligned*.fasta
    do  appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
        if    [ ! -f "$appendix"_"$trimappendix".fasta ]
        then  echo "trimming alignment $a with gap threshold $g"
              sed -i 's/ /_/g' $a
              trimal -in $a   \
                     -out data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".fasta \
                     -gt ."$g" \
                     -htmlout data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".html &
        fi
    done
done
wait
conda deactivate

(phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) (phylogenetics) trimming alignment data/alignments_raw/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi.fasta with gap threshold 1
[1] 486823
trimming alignment data/alignments_raw/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi.fasta with gap threshold 2
[2] 486831
trimming alignment data/alignments_raw/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi.fasta with gap threshold 3
[3] 486838
trimming alignment data/alignments_raw/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi.fasta with gap threshold 4
[4] 486845
trimming alignment data/alignments_raw/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi.fasta with gap threshold 5
[5] 486852
(phylogenetics) [1]   Done                    trimal -in $a -out data/alignments_trimmed/"$inseq"_"$appendix"_"$trimappendix".fasta -gt ."$g" -htmlout data/alignments_trimmed/"$inseq"_"$appendix"_"$t

In [21]:
conda activate jalview
for   i in data/alignments_trimmed/*.fasta
do    prefix=$(echo $i | sed 's/\.fasta//')
      if    [ ! -f $prefix.png ]
      then  jalview -nodisplay \
                    -open $prefix.fasta \
                    -colour CLUSTAL \
                    -png  $prefix.png > /dev/null 2> /dev/null &
      fi
done
wait
conda deactivate

(jalview) [1] 487881
[2] 487888
[3] 487898
[4] 487915
[5] 487922
[6] 487926
[7] 487958
[8] 487972
[9] 487981
[10] 487993
[11] 488004
[12] 488044
[13] 488056
[14] 488078
[15] 488087
[16] 488100
[17] 488140
[18] 488165
[19] 488191
[20] 488200
[21] 488212
[22] 488229
[23] 488246
[24] 488259
[25] 488268
[26] 488288
[27] 488298
[28] 488307
[29] 488317
(jalview) [1]   Done                    jalview -nodisplay -open $prefix.fasta -colour CLUSTAL -png $prefix.png > /dev/null 2> /dev/null
[5]   Done                    jalview -nodisplay -open $prefix.fasta -colour CLUSTAL -png $prefix.png > /dev/null 2> /dev/null
[8]   Done                    jalview -nodisplay -open $prefix.fasta -colour CLUSTAL -png $prefix.png > /dev/null 2> /dev/null
[9]   Done                    jalview -nodisplay -open $prefix.fasta -colour CLUSTAL -png $prefix.png > /dev/null 2> /dev/null
[11]   Done                    jalview -nodisplay -open $prefix.fasta -colour CLUSTAL -png $prefix.png > /dev/null 2> /dev/null
[12] 

The Final alignment looks like this, quite happy with it. 
### gt 1
![image.png](data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt1.png)

### gt 2
![image.png](data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt2.png)

### gt 3
![image.png](data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt3.png)

### gt 4
![image.png](data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt4.png)

### gt 5
![image.png](data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt5.png)

# tree inference
Now let's get to tree inference, choosing a fast bootstrapping method

In [2]:
inseq=MIKC_orthogroup_selection-basal-v7_guide-v4
ls data/alignments_trimmed/"$inseq"_aligned*fasta

data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt1.fasta
data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt2.fasta
data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt3.fasta
data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt4.fasta
data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt5.fasta


In [23]:
conda activate phylogenetics
for a in data/alignments_trimmed/"$inseq"_aligned*gt*.fasta
do  #iqpendix='iqtree-b100'
    iqpendix='iqtree-bb2000-alrt2000'

    file_appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
    
    if   [ ! -d    analyses/"$inseq"_trees/"$file_appendix" ]
    then echo "Making a directory $file_appendix to store trees (name based on alignment filename)"
         mkdir -p  analyses/"$inseq"_trees/"$file_appendix" 
    fi

    iqprefix=analyses/"$inseq"_trees/"$file_appendix"/"$inseq"_"$file_appendix"_"$iqpendix"
    if   [ ! -f "$iqprefix".treefile ]
    then echo "making a tree of file $a"
         nice iqtree -s $a \
                     -m 'MFP' \
                     -bb 2000 -alrt 2000 \
                     -nt AUTO \
                     -ntmax $(nproc)  \
                     -pre  "$iqprefix" \
                     2>   "$iqprefix".stderr \
                      >    "$iqprefix".stdout
    #cat "$iqprefix".log | mail -s "IQtree_run $a" laura
    fi
done
conda deactivate

(phylogenetics) Making a directory aligned-mafft-einsi_trim-gt1 to store trees (name based on alignment filename)
making a tree of file data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt1.fasta
Making a directory aligned-mafft-einsi_trim-gt2 to store trees (name based on alignment filename)
making a tree of file data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt2.fasta
Making a directory aligned-mafft-einsi_trim-gt3 to store trees (name based on alignment filename)
making a tree of file data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt3.fasta
Making a directory aligned-mafft-einsi_trim-gt4 to store trees (name based on alignment filename)
making a tree of file data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt4.fasta
Making a directory aligned-mafft-einsi_trim-gt5 to store trees (name based on alignment file

In [27]:
ls analyses/"$inseq"_trees/*/*.log

analyses/MIKC_orthogroup_selection-basal-v7_guide-v4_trees/aligned-mafft-einsi_trim-gt1/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt1_iqtree-bb2000-alrt2000.log
analyses/MIKC_orthogroup_selection-basal-v7_guide-v4_trees/aligned-mafft-einsi_trim-gt2/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt2_iqtree-bb2000-alrt2000.log
analyses/MIKC_orthogroup_selection-basal-v7_guide-v4_trees/aligned-mafft-einsi_trim-gt3/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt3_iqtree-bb2000-alrt2000.log
analyses/MIKC_orthogroup_selection-basal-v7_guide-v4_trees/aligned-mafft-einsi_trim-gt4/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt4_iqtree-bb2000-alrt2000.log
analyses/MIKC_orthogroup_selection-basal-v7_guide-v4_trees/aligned-mafft-einsi_trim-gt5/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt5_iqtree-bb2000-alrt2000.log


In [28]:
grep 'Best-fit model' analyses/"$inseq"_trees/*/*.log

analyses/MIKC_orthogroup_selection-basal-v7_guide-v4_trees/aligned-mafft-einsi_trim-gt1/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt1_iqtree-bb2000-alrt2000.log:Best-fit model: JTT+R6 chosen according to BIC
analyses/MIKC_orthogroup_selection-basal-v7_guide-v4_trees/aligned-mafft-einsi_trim-gt2/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt2_iqtree-bb2000-alrt2000.log:Best-fit model: JTT+R8 chosen according to BIC
analyses/MIKC_orthogroup_selection-basal-v7_guide-v4_trees/aligned-mafft-einsi_trim-gt3/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt3_iqtree-bb2000-alrt2000.log:Best-fit model: JTT+R7 chosen according to BIC
analyses/MIKC_orthogroup_selection-basal-v7_guide-v4_trees/aligned-mafft-einsi_trim-gt4/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt4_iqtree-bb2000-alrt2000.log:Best-fit model: JTT+R6 chosen according to BIC
analyses/MIKC_orthogroup_selection-basal-v7_guide-v4_trees/a

In [ ]:
tail -n 30 $iqprefix.log

In [ ]:
grep 'Best-fit model' $iqprefix.log

## Conclusions

Walking through these trees quickly, I like the first and the latter the most. (-gt .1 and -gt .5).

Here is a snapshot of both:

### 1
![gt .1](analyses/MIKC_orthogroup_selection-basal-v7_guide-v4_trees/aligned-mafft-einsi_trim-gt1/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt1_iqtree-bb2000-alrt2000.png)

### 5
![gt .5](analyses/MIKC_orthogroup_selection-basal-v7_guide-v4_trees/aligned-mafft-einsi_trim-gt5/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt5_iqtree-bb2000-alrt2000.png)

Regardless which one you pick, branch lenghts separating the major clades are still very short but those nodes tend to be better supported in the `-gt .5` tree.

Also the homeotic B genes end up where they should, and the algae, bryophyte, lycophyte, monilophyte speciation patter surelly makes that I trust this tree better.

Visually, I miss the algal outgroup, it communicated the strength of this root much better.

I wonder wether it is justified and desireable to remove more reduculously long branches and to reduce the amount of input species. 
Looking at the list of species, I don't want to remove any, but looking at the tree, I think it would do well with a couple less. 

Let's make a normal bootstrap version of the `-gt .5 tree` and see if it is consistent with the UFbootstrap one. 
I'll resuse the model fitted before, since the alignment is unchanged.

In [ ]:
a=data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt5.fasta

iqpendix='iqtree-LGR7-b1000'
#iqpendix='iqtree-MFP-bb2000-alrt2000'

echo "making a tree of file $a"

file_appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
echo "Making a directory $file_appendix to store trees (name based on alignment filename)"

if   [ ! -d    analyses/"$inseq"_trees/"$file_appendix" ]
then mkdir -p  analyses/"$inseq"_trees/"$file_appendix" 
fi

iqprefix=analyses/"$inseq"_trees/"$file_appendix"/"$inseq"_"$file_appendix"_"$iqpendix"

if   [ ! -f "$iqprefix".treefile ]
then echo "Making $iqprefix.treefile"
     nice iqtree -s $a    \
                 -m LG+R7  \
                 -b 1000  \
                 -nt AUTO -ntmax $(nproc) \
                 -pre  "$iqprefix" \
                 2>   "$iqprefix".stderr \
                 >    "$iqprefix".stdout && cat "$iqprefix".log | mail -s IQtree_run laura
else echo "NOT OVERWRITING $iqprefix.treefile"
fi

making a tree of file data/alignments_trimmed/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt5.fasta
Making a directory aligned-mafft-einsi_trim-gt5 to store trees (name based on alignment filename)
Making analyses/MIKC_orthogroup_selection-basal-v7_guide-v4_trees/aligned-mafft-einsi_trim-gt5/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt5_iqtree-LGR7-b1000.treefile


## observations

My suspisions were correct, in the UFbootstrap tree there is barely support for the important clades, let alone their relative position to each other. 
In the nonparametric tree, it turns out those fast methods were a substantial over-estimation.
Also the support at the base of several important clades is practically absent. 
[itol](https://itol.embl.de/tree/1312115999190001598359608#)

Most of the topoply is reproducible in the many trees made, but the signal is still weak. 
![](analyses/MIKC_orthogroup_selection-basal-v7_guide-v4_trees/aligned-mafft-einsi_trim-gt5/MIKC_orthogroup_selection-basal-v7_guide-v4_aligned-mafft-einsi_trim-gt5_iqtree-LGR7-b1000.png)